In [145]:
version = 7

In [12]:
import json

f_raw = "batch-2.jsonl"

with open(f_raw) as fin:
    data = [json.loads(line) for line in fin]
print(len(data))

1200


In [13]:
outputs = []
err_cnt_dist = dict()
err_type_dist = dict()
for idx,line in enumerate(data):
    # print(idx)
    
    flag = False
    if "CLASSIFICATION_JOB_2" in line["latestLabel"]["jsonResponse"]:
        flags = line["latestLabel"]["jsonResponse"]["CLASSIFICATION_JOB_2"]["categories"]
        for i in range(len(flags)):
            if line["latestLabel"]["jsonResponse"]["CLASSIFICATION_JOB_2"]["categories"][i]["name"] == "ERRORS_IN_THE_CORRECT_OUTPUT":
                flag = True
            elif line["latestLabel"]["jsonResponse"]["CLASSIFICATION_JOB_2"]["categories"][i]["name"] == "THE_CONTEXT_IS_VERY_COMPLEX_IM_NOT_SURE_OF_MY_ANSWER":
                flag = True
    if flag:
        continue

    labels = line["latestLabel"]["jsonResponse"]["CLASSIFICATION_JOB_0"]["categories"]
    feedback = []
    error_types = []
    for label in labels:
        feedback.extend([i["text"] for i in label["children"].values()])
        error_types.append(label["categoryName"])
        
    # priorities
    priorities = ['Coherence and Deduction', 'Veracity', 'Arithmetic', 'Commonsense'] #'Consistency with Context', 'Redundancy']
    
    if len(feedback) > 1:
        new_feedback = []
        new_error_type = []
        
        for priority in priorities:
            for type_idx in range(len(feedback)):
                if error_types[type_idx] == priority:
                    new_error_type.append(priority)
                    new_feedback.append(feedback[type_idx])
        
        feedback = new_feedback
        error_types = new_error_type
        
    if len(feedback) < 1: continue
                    
    
    # remove candidate
    for item_id in range(len(feedback)):
        if "victory is an election result in which the victorious candidate" in feedback[item_id]: 
            feedback[item_id] = feedback[item_id].replace("Candidates", "The answer")
        feedback[item_id] = feedback[item_id].replace("the candidate output", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate output's answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate's answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("The candidate output", "The answer")
        feedback[item_id] = feedback[item_id].replace("The candidate output'answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("The candidate'answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("the candidate had the answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidates answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate does not answer the question", "this does not answer the question")
        
        feedback[item_id] = feedback[item_id].replace("The candidates", "The answer")
        feedback[item_id] = feedback[item_id].replace("The candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("The candidate's answer", "The answer")
        
        feedback[item_id] = feedback[item_id].replace("The output candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("the output candidate", "the answer")
        
        feedback[item_id] = feedback[item_id].replace("candidate's answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate output", "the answer")
        feedback[item_id] = feedback[item_id].replace("candidate output", "the answer")
        feedback[item_id] = feedback[item_id].replace("Candidate output", "The answer")
        feedback[item_id] = feedback[item_id].replace("Candidate Output", "The answer")
        feedback[item_id] = feedback[item_id].replace("candidate answers", "it answers")
        feedback[item_id] = feedback[item_id].replace("Th candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("Output Candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("Output candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("Candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("Candidates", "The answer")
        
        feedback[item_id] = feedback[item_id].replace("candidate", "the answer")
        
        feedback[item_id] = feedback[item_id].replace("The answer answers ", "The answer says ")
        feedback[item_id] = feedback[item_id].replace("the answer answers ", "the answer says ")
        feedback[item_id] = feedback[item_id].replace("The answer answered ", "The answer sayed ")
        feedback[item_id] = feedback[item_id].replace("the answer answered ", "the answer sayed ")
        feedback[item_id] = feedback[item_id].replace("The answer answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("The answer in the answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("the answers' answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("The answer's answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("The answer’s answer", "The answer")
        
        answer_loc_1 = feedback[item_id].find("answer")
        answer_loc_2 = feedback[item_id][answer_loc_1+5:].find("answer")
        if answer_loc_2 > -1 and answer_loc_2 < 10:
            print(feedback[item_id])
            print(" ")
        
    
    my_key = "candidate"
    for item in feedback:
        loc = -1
        if my_key in item:
            loc = item.find(my_key)
        
        if loc > -1:
            print(item[loc:loc+155])
        
    if len(feedback) in err_cnt_dist:
        err_cnt_dist[len(feedback)] += 1
    else:
        err_cnt_dist[len(feedback)] = 1
        
    for err_type in error_types:
        if err_type in err_type_dist:
            err_type_dist[err_type] += 1
        else:
            err_type_dist[err_type] = 1
        
    question = line["metadata"]["context"]
    answer = line["metadata"]["output_candidate"]
    
    critique = " ".join(feedback)
    if "correct output" in critique or "Correct output" in critique or "Correct Output" in critique:
#         print("prompt_id:", line["metadata"]["prompt_id"])
        continue
    
    outputs.append({"question": question,
                  "answer": answer,
                  "feedback": feedback,
                  "error_types": error_types})
print(len(outputs))
print(err_cnt_dist)
print(err_type_dist)

The answer cannot answer the question because the context does not provide enough information to come to a justifiable conclusion.
The rule does not state is the cat can see the cow.  
 
The answer never answers question. It says "Therefore, while it is possible that the individual will be happy after finding out they don't have cancer, it is also possible that they will be upset or unsettled by the experience". The correct answer is happiness. 
 
The answer only answered  how much geckos Brandon sold in the previous year but the question was how much he sold altogether so the answer should be what he sold this year plus what he sold the previous year.
 
1064
{1: 1033, 2: 32}
{'Coherence and Deduction': 474, 'Consistency with Context': 105, 'Redundancy': 106, 'No error': 218, 'Arithmetic': 69, 'Commonsense': 60, 'Veracity': 65}


In [23]:
version = 3

processed_outputs = []
err_type_dist = dict()

bad_count_redundancy = 10
bad_count_consistent = 10

for idx, output in enumerate(outputs):
    
    if len(output["feedback"]) == 1:
        if output["error_types"][0] == "No error":
            critique = output["feedback"][0]
        elif " correct" in output["feedback"][0]:
            critique = output["feedback"][0]
        elif output["error_types"][0] in ["Redundancy", "Consistency with Context"]:
            critique = output["feedback"][0]
        else:
            critique = "" + output["feedback"][0]
            
#         if output["error_types"][0] in ["Redundancy", "Consistency with Context"]:
#             continue
        
        if output["error_types"][0] == "Redundancy":
            if bad_count_redundancy > 0:
                bad_count_redundancy -= 1
            else:
                continue
        if output["error_types"][0] == "Consistency with Context":
            if bad_count_consistent > 0:
                bad_count_consistent -= 1
            else:
                continue
                
    elif len(output["feedback"]) == 2:
        critique = "Firstly, " + output["feedback"][0] + " Secondly, " + output["feedback"][1]
    elif len(output["feedback"]) == 3:
        critique = "Firstly, " + output["feedback"][0] + " Secondly, " + output["feedback"][1] + \
        " Thirdly, " + output["feedback"][2]
    
    if "in which the victorious candidate or party wins by an overwhelming margin. Therefore, the vote could not" in critique: continue
    
    for err_type in output["error_types"]:
        if err_type in err_type_dist:
            err_type_dist[err_type] += 1
        else:
            err_type_dist[err_type] = 1
    
#     if "Veracity" in output["error_types"]:
#         print(critique)
    if idx % 70 == 0:
        print(critique)
        print(output["error_types"])
        print()
    processed_outputs.append({"id":idx,"question":output["question"],"answer":output["answer"],
                "feedback":critique,"meta":{"domain":"human annotation"}})
    



print(len(processed_outputs))

with open(f"./v2/v2.{version}.jsonl", "w") as fout:
    for output in processed_outputs:
        fout.write(f"{json.dumps(output)}\n")

The answer fails to start with the right step.
The answer divides 11 by 2 initially which leads to a wrong answer.
The right first step would be 11+3 to get 14 then divide 14 by 2. The correct answer is therefore 7-3=4
['Coherence and Deduction']

The answer seems to miss the part of the context that mentions an indoor sport. The right answer should be bowling alley, as bowling is an indoor sport with seats behind a scoreboard.
['Coherence and Deduction']

The answer is correct.
['No error']

The answer is correct.
['No error']

The output provides a totally incoherent answer in that it does not respond to what is in the context at all. It's supposed to address whether Chelsea injured Arsenal. The right answer is that Chelsea did not injure Arsenal, as Arsenal's injuries were present before the match versus Chelsea.
['Coherence and Deduction']

The answer is correct until it mentions sieving and filtration. Filtration is not used for larger stuff like fruits. 
['Commonsense']

It was j

In [24]:
print(err_type_dist)

{'Coherence and Deduction': 473, 'Consistency with Context': 10, 'Redundancy': 10, 'No error': 218, 'Arithmetic': 69, 'Commonsense': 60, 'Veracity': 65}
